### Imports

In [182]:
import torch
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
from tqdm import tqdm
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from TorchCRF import CRF

%run updatePreprocessing.ipynb

[['ق', 'و', 'ل', 'ه'], ['ل', 'ع', 'د', 'م'], ['م', 'ا'], ['ت', 'ت', 'ع', 'ل', 'ق'], ['إ', 'ل', 'خ'], ['أ', 'ي'], ['ا', 'ل', 'و', 'ص', 'ي', 'ة'], ['ق', 'و', 'ل', 'ه'], ['م', 'ا'], ['م', 'ر'], ['أ', 'ي'], ['ق', 'ب', 'ي', 'ل'], ['ق', 'و', 'ل'], ['ا', 'ل', 'م', 'ت', 'ن'], ['ل', 'غ', 'ت'], ['و', 'ل', 'و'], ['ا', 'ق', 'ت', 'ص', 'ر'], ['ع', 'ل', 'ى'], ['أ', 'و', 'ص', 'ي', 'ت'], ['ل', 'ه'], ['ب', 'ش', 'ا', 'ة'], ['أ', 'و'], ['أ', 'ع', 'ط', 'و', 'ه'], ['ش', 'ا', 'ة'], ['و', 'ل', 'ا'], ['غ', 'ن', 'م'], ['ل', 'ه'], ['ع', 'ن', 'د'], ['ا', 'ل', 'م', 'و', 'ت'], ['ه', 'ل'], ['ت', 'ب', 'ط', 'ل'], ['ا', 'ل', 'و', 'ص', 'ي', 'ة'], ['أ', 'و'], ['ي', 'ش', 'ت', 'ر', 'ى'], ['ل', 'ه'], ['ش', 'ا', 'ة'], ['و', 'ي', 'ؤ', 'خ', 'ذ'], ['م', 'ن'], ['ق', 'و', 'ل', 'ه'], ['ا', 'ل', 'آ', 'ت', 'ي'], ['ك', 'م', 'ا'], ['ل', 'و'], ['ل', 'م'], ['ي', 'ق', 'ل'], ['م', 'ن'], ['م', 'ا', 'ل', 'ي'], ['و', 'ل', 'ا'], ['م', 'ن'], ['غ', 'ن', 'م', 'ي'], ['أ', 'ن', 'ه', 'ا'], ['ل', 'ا'], ['ت', 'ب', 'ط', 'ل'], ['و', 'ع', 'ب', 'ا', 'ر',

### Constants

In [183]:
EMBEDDING_DIM = 300
HIDDEN_SIZE = 512
NUM_LAYERS = 2
NUM_EPOCHS = 10
LEARNING_RATE = 0.01
BATCH_SIZE = 256
VOCAB_SIZE = len(basic_arabic_letters) + 1
LABELS_SIZE = len(DIACRITICS)

TRAIN_PATH = "../dataset/train.txt"
VAL_PATH = "../dataset/val.txt"
LSTM_PATH="./models/lstm.pth"
RNN_PATH="./models/rnn.pth"
CNN_PATH = "./models/cnn.pth"
CRF_PATH = "./models/crf.pth"

### Model building

### RNN

In [184]:
class RNN(nn.Module):
    def __init__(self, vocab_size=VOCAB_SIZE, n_classes=LABELS_SIZE, embedding_dim=EMBEDDING_DIM, hidden_size=HIDDEN_SIZE, num_layers=NUM_LAYERS):
        """
        The constructor of our RNN model
        Inputs:
        - vacab_size: the number of unique characters
        - embedding_dim: the embedding dimension
        - n_classes: the number of final classes (diacritics)
        """
        super(RNN, self).__init__()

        # (1) Create an embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # (2) Create an LSTM layer with hidden size = hidden_size and batch_first = True
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True)
        
        # (3) Create a linear layer with number of neorons = n_classes
        self.linear = nn.Linear(hidden_size * 2, n_classes)

    def forward(self, sentences):
        """
        This function does the forward pass of our model
        Inputs:
        - sentences: tensor of shape (batch_size, max_length)

        Returns:
        - final_output: tensor of shape (batch_size, max_length, n_classes)
        """

        final_output = None
        
        embeddings = self.embedding(sentences)
        lstm_out, _ = self.lstm(embeddings)
        output = self.linear(lstm_out)
        # final_output = F.softmax(output, dim=1)
        return output

### CNN

In [185]:
class CNN(nn.Module):
    def __init__(self, vocab_size=VOCAB_SIZE, num_classes=LABELS_SIZE, embedding_dim=EMBEDDING_DIM, hidden_size=HIDDEN_SIZE, num_layers=NUM_LAYERS):
        super(CNN, self).__init__()
        
        # Embedding Layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # Conv Layer
        self.conv1d = nn.Conv1d(embedding_dim, 128, kernel_size=3, padding=1)
        
        # LSTM Layer
        self.lstm = nn.LSTM(128, hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True)
        
        # Linear Layer
        self.linear = nn.Linear(2 * hidden_size, num_classes)

    def forward(self, sentences):
        embeddings = self.embedding(sentences)
        
        conv_out = self.conv1d(embeddings.permute(0, 2, 1))
        conv_out = F.relu(conv_out)
        conv_out = conv_out.permute(0, 2, 1)
        
        lstm_out, _ = self.lstm(conv_out)
        
        output = self.linear(lstm_out)

        return output

### CRF

In [186]:
class LSTM_CRF(nn.Module):
    def __init__(self, vocab_size=VOCAB_SIZE, n_classes=LABELS_SIZE, embedding_dim=EMBEDDING_DIM, hidden_size=HIDDEN_SIZE, num_layers=NUM_LAYERS):
        super(LSTM_CRF, self).__init__()

        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # LSTM layer
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True)

        # Linear layer
        self.linear = nn.Linear(hidden_size * 2, n_classes)

        # CRF layer
        self.crf = CRF(n_classes)  # Place the CRF layer after the linear layer

    def forward(self, sentences):
        embeddings = self.embedding(sentences)
        lstm_out, _ = self.lstm(embeddings)
        output = self.linear(lstm_out)
        return output  # Return raw output for CRF loss calculation

    def predict(self, sentences):
        output = self.forward(sentences)
        predictions = self.crf.decode(output)
        return predictions


### Train

In [187]:
from torch.optim.lr_scheduler import StepLR

def train(model, path, train_dataset, train_labels, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, learning_rate=LEARNING_RATE):
    """
    This function implements the training logic
    Inputs:
    - model: the model to be trained
    - train_dataset: the training set
    - batch_size: integer represents the number of examples per step
    - epochs: integer represents the total number of epochs (full training pass)
    - learning_rate: the learning rate to be used by the optimizer
    """

    # (1) create the dataloader of the training set (make the shuffle=True)
    tensor_train_dataset = TensorDataset(train_dataset, train_labels)
    train_dataloader = DataLoader(tensor_train_dataset, batch_size=batch_size, shuffle=True)

    # (2) make the criterion cross entropy loss
    criterion = nn.CrossEntropyLoss()

    # (3) create the optimizer (Adam)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    # (4) create a learning rate scheduler (optional but recommended)
    scheduler = StepLR(optimizer, step_size=5, gamma=0.1)  # Adjust parameters as needed

    # GPU configuration
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()
    
    best_accuracy = 0.0
    for epoch_num in range(epochs):
        total_acc_train = 0
        total_loss_train = 0
        
        for train_input, train_label in tqdm(train_dataloader):
            # Zero your gradients
            optimizer.zero_grad()

            # Move the train input to the device
            train_label = train_label.to(device)

            # Move the train label to the device
            train_input = train_input.to(device)

            # Do the forward pass
            output = model(train_input).float()

            # Loss calculation
            batch_loss = criterion(output.view(-1, output.shape[-1]), train_label.view(-1))

            # Append the batch loss to the total_loss_train
            total_loss_train += batch_loss.item()
            
            # Calculate the batch accuracy (just add the number of correct predictions)
            # Compare predicted diacritic with true diacritic and count correct predictions
            correct_predictions = (output.argmax(dim=2) == train_label)

            # Calculate accuracy for the current batch
            acc = correct_predictions.sum().item()
            total_acc_train += acc

            # Do the backward pass
            batch_loss.backward()

            # Update the weights with your optimizer
            optimizer.step()     
        # Step the learning rate scheduler
        scheduler.step()
        # Calculate the epoch loss
        epoch_loss = total_loss_train / len(train_dataset)

        # Calculate the accuracy
        epoch_acc = total_acc_train / (len(train_dataset) * len(train_dataset[0]))

        print(f'Epochs: {epoch_num + 1} | Train Loss: {epoch_loss} \
            | Train Accuracy: {epoch_acc}\n')
        
        if epoch_acc > best_accuracy:
            best_accuracy = epoch_acc
            torch.save(model.state_dict(), path)
            print(f'Saved the best model with accuracy: {best_accuracy} to {path}\n')


### Prepare Data

In [188]:
corpus=  readFile(TRAIN_PATH)

x_train = []
y_train = []

for sentence in corpus[:1000]:
	# Clean each sentence in the corpus
	# Get the char list for each word in the sentence and its corresponding diacritics
	char_list, diacritics_list = separate_words_and_diacritics(sentence.strip())

	x_train.append(char_list)
	y_train.append(diacritics_list)

X_train_padded = [torch.tensor([char_to_index[char] for char in word]) for sentence in x_train for word in sentence]
X_train_padded = pad_sequence(X_train_padded, batch_first=True)

y_train_padded = [torch.tensor([diacritic_to_index[char] for char in word]) for sentence in y_train for word in sentence]
y_train_padded = pad_sequence(y_train_padded, batch_first=True)

### Execute

In [189]:
def run_RNN():
    model=RNN()
    print(model)
    train(model, LSTM_PATH, X_train_padded, y_train_padded)
    
def run_CNN():
    model=CNN()
    print(model)
    train(model, CNN_PATH, X_train_padded, y_train_padded)

def run_CRF():
    model=LSTM_CRF()
    print(model)
    train(model, CRF_PATH, X_train_padded, y_train_padded)

In [ ]:
# run_RNN()
run_CNN()
# run_CRF()